# COVID-19 In Washington State
> Tracking coronavirus cases by county, age and gender across Washington

- comments: true
- author: Bilal Tahir
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-wa/

In [12]:
#hide
import numpy as np
import pandas as pd
import geopandas as gpd
import json
from jinja2 import Template
import altair as alt
import gcsfs
from IPython.display import HTML

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
#hide_input
HTML("<div>Hello World!</div>")

In [2]:
#hide
url = 'gs://covid_wa/dfCasesByCounty.csv'
df = pd.read_csv(url)

df.head()

,County,Confirmed,Deaths
0,Benton,7,2
1,Chelan,3,0
2,Clallam,2,0
3,Clark,9,3
4,Columbia,1,0


In [6]:
#hide
counties_geo = gpd.read_file("WA-53-washington-counties.json")
counties_geo.head()

,id,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,None,53,063,01529225,0500000US53063,53063,Spokane,06,4568197031,43789502,"POLYGON ((-117.82144 47.82584, -117.69950 47.8..."
1,None,53,041,01531927,0500000US53041,53041,Lewis,06,6223223859,86636988,"POLYGON ((-123.37212 46.79151, -123.20424 46.7..."
2,None,53,025,01531924,0500000US53025,53025,Grant,06,6939890129,289550821,"POLYGON ((-120.00743 47.22020, -120.00566 47.3..."
3,None,53,051,01529157,0500000US53051,53051,Pend Oreille,06,3626035232,65404066,"POLYGON ((-117.42912 48.99974, -117.26831 48.9..."
4,None,53,023,01533500,0500000US53023,53023,Garfield,06,1840672367,19490299,"POLYGON ((-117.85325 46.62453, -117.75075 46.6..."


In [23]:
#hide
geo_df = counties_geo.merge(df, left_on='NAME', right_on='County', how='left')
geo_df = geo_df[geo_df.geometry != None]
geo_df = geo_df[['NAME', 'Confirmed', 'Deaths', 'geometry']]
# geo_df["Confirmed"] = pd.to_numeric(geo_df["Confirmed"], downcast='integer')
geo_df = geo_df.fillna(0)
geo_df.head()

,NAME,Confirmed,Deaths,geometry
0,Spokane,16.0,0,"POLYGON ((-117.82144 47.82584, -117.69950 47.8..."
1,Lewis,2.0,0,"POLYGON ((-123.37212 46.79151, -123.20424 46.7..."
2,Grant,11.0,1,"POLYGON ((-120.00743 47.22020, -120.00566 47.3..."
3,Pend Oreille,0.0,0,"POLYGON ((-117.42912 48.99974, -117.26831 48.9..."
4,Garfield,0.0,0,"POLYGON ((-117.85325 46.62453, -117.75075 46.6..."


In [8]:
#hide
# convert to geopandas dataframe
gdf = gpd.GeoDataFrame(
    geo_df, geometry=geo_df['geometry'])

gdf.head()

,NAME,Confirmed,Deaths,geometry
0,Spokane,16.0,0,"POLYGON ((-117.82144 47.82584, -117.69950 47.8..."
1,Lewis,2.0,0,"POLYGON ((-123.37212 46.79151, -123.20424 46.7..."
2,Grant,11.0,1,"POLYGON ((-120.00743 47.22020, -120.00566 47.3..."
3,Pend Oreille,0.0,0,"POLYGON ((-117.42912 48.99974, -117.26831 48.9..."
4,Garfield,0.0,0,"POLYGON ((-117.85325 46.62453, -117.75075 46.6..."


In [9]:
#hide
gdf['centroid_lon'] = gdf['geometry'].centroid.x
gdf['centroid_lat'] = gdf['geometry'].centroid.y
gdf.head()

,NAME,Confirmed,Deaths,geometry,centroid_lon,centroid_lat
0,Spokane,16.0,0,"POLYGON ((-117.82144 47.82584, -117.69950 47.8...",-117.405248,47.621177
1,Lewis,2.0,0,"POLYGON ((-123.37212 46.79151, -123.20424 46.7...",-122.393496,46.577689
2,Grant,11.0,1,"POLYGON ((-120.00743 47.22020, -120.00566 47.3...",-119.450183,47.206904
3,Pend Oreille,0.0,0,"POLYGON ((-117.42912 48.99974, -117.26831 48.9...",-117.274313,48.532324
4,Garfield,0.0,0,"POLYGON ((-117.85325 46.62453, -117.75075 46.6...",-117.546092,46.431514


In [13]:
#hide
choro_json = json.loads(gdf.to_json())
choro_data = alt.Data(values=choro_json['features'])

In [28]:
#hide_input
def gen_map(geodata, color_column, title, tooltip):
    '''Generates DC ANC map with population choropleth and ANC labels'''
    # Add Base Layer
    base = alt.Chart(geodata, title = title).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=800,
        height=800,
    )
    # Add Choropleth Layer
    choro = alt.Chart(geodata).mark_geoshape(
        stroke='black'
    ).properties(
        width=800,
        height=800
     ).encode(
        alt.Color(color_column, 
                  type='quantitative', 
                  scale=alt.Scale(scheme='yelloworangered'),
                  title = "COVIDL"),
        tooltip=tooltip,
    )
    # Add Labels Layer
    labels = alt.Chart(geodata).mark_text(baseline='top'
     ).encode(
         longitude='properties.centroid_lon:Q',
         latitude='properties.centroid_lat:Q',
         text='properties.NAME:O',
         size=alt.value(8),
         opacity=alt.value(1)
     )

    return base + choro + labels

wa_map = gen_map(geodata=choro_data, color_column='properties.Confirmed', title='COVID', 
                       tooltip=[alt.Tooltip('properties.NAME:O', title='County'),
                                alt.Tooltip('properties.Confirmed:Q', title='Confirmed Cases'),
                                alt.Tooltip('properties.Deaths:Q', title='Deaths')])
wa_map

alt.LayerChart(...)

In [41]:
#hide
states_data = 'https://vega.github.io/vega-datasets/data/us-10m.json'
states = alt.topo_feature(states_data, feature='states')
selector = alt.selection_single(empty='none', fields=['state'], nearest=True, init={'state':'WA'})

curr_date = state_df.date.max().date().strftime('%Y-%m-%d')
dmax = (us_daily_df.date.max() + pd.DateOffset(days=3))
dmin = us_daily_df.date.min()

# US states background
background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=400
).project('albersUsa')


points = alt.Chart(state_df).mark_circle().encode(
    longitude='long:Q',
    latitude='lat:Q',
    size=alt.Size('confirmed_count:Q', title= 'Number of Confirmed Cases'),
    color=alt.value('steelblue'),
    tooltip=['state:N','confirmed_count:Q']
).properties(
    title=f'Total Confirmed Cases by State as of {curr_date}'
).add_selection(selector)


final_chart = alt.vconcat(
    background + points, 
).resolve_scale(
    color='independent',
    shape='independent',
).configure(
    padding={'left':20, 'bottom':40}
).configure_axis(
    labelFontSize=10,
    labelPadding=10,
    titleFontSize=12,
).configure_view(
     stroke=None
)

In [42]:
#hide_input
final_chart

alt.VConcatChart(...)